<a href="https://colab.research.google.com/github/JarekMaleszyk/data-science-project-sandbox/blob/main/apache_spark_lrex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import pyspark
except:
  !pip install pyspark -q
  import pyspark
finally:
  from pyspark.sql import SparkSession

In [2]:
sparkSession = SparkSession.builder.appName("lrex").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 08:17:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/12 08:17:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
!pwd

/home/hans/dev/data-science-project-sandbox


In [7]:
import os.path
FILE_PATH1 = "./content/csvdata1/Ecommerce_Customers.csv"

if not os.path.isfile(FILE_PATH1):
  !rm -rf /content/csvdata1
  !wget -P "./content/csvdata1/" "https://raw.githubusercontent.com/JarekMaleszyk/data-science-project-sandbox/refs/heads/main/Ecommerce_Customers.csv"

--2024-11-12 08:18:40--  https://raw.githubusercontent.com/JarekMaleszyk/data-science-project-sandbox/refs/heads/main/Ecommerce_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86871 (85K) [text/plain]
Saving to: ‘./content/csvdata1/Ecommerce_Customers.csv’

Ecommerce_Customers 100%[===================>]  84.83K  --.-KB/s    in 0.1s    

2024-11-12 08:18:41 (617 KB/s) - ‘./content/csvdata1/Ecommerce_Customers.csv’ saved [86871/86871]



In [8]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = sparkSession.read.csv(FILE_PATH1, sep=",", header=True, inferSchema=True)

In [10]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length',
               'Time on App',
               'Time on Website',
               'Length of Membership'],
    outputCol="features"
)

In [13]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
output.show(1)

+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avatar|Avg Session Length|      Time on App|  Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|Violet| 34.49726772511229|12.65565114916675|39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+
only showing top 1 row



In [16]:
final_data = output.select("features", "Yearly Amount Spent")

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [19]:
lr_model = lr.fit(train_data)

24/11/12 08:19:28 WARN Instrumentation: [859ba907] regParam is zero, which might cause numerical instability and overfitting.


In [20]:
test_result = lr_model.evaluate(test_data)

In [21]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  12.704249362195242|
|  -9.785605982430639|
| -3.1005795654774033|
|   -3.12604344242294|
|-0.19770825343573506|
|   5.064304058140465|
|  1.6860430120671026|
|  -3.661555418798571|
|  -5.443232011745067|
|   3.676604752016658|
| -1.4565651713672878|
|   18.90387112961912|
| -0.6925968853276459|
|   3.866588675902733|
|   8.658416664814183|
|   9.071456655095574|
| -1.8300361170427095|
| -2.0349628461623297|
|  -4.375914443771023|
|  -8.084871560852605|
+--------------------+
only showing top 20 rows



In [22]:
test_result.rootMeanSquaredError

10.13583435793872

In [23]:
test_result.r2

0.98488216700537

In [26]:
import os.path
FILE_PATH2 = "./content/csvdata2/cruise_ship_info.csv"

if not os.path.isfile(FILE_PATH2):
  !rm -rf /content/csvdata2
  !wget -P "./content/csvdata2/" "https://raw.githubusercontent.com/LeondraJames/Hyundai-Cruise-Ship-Crew-Prediction/refs/heads/master/cruise_ship_info.csv"

--2024-11-12 08:19:52--  https://raw.githubusercontent.com/LeondraJames/Hyundai-Cruise-Ship-Crew-Prediction/refs/heads/master/cruise_ship_info.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8734 (8.5K) [text/plain]
Saving to: ‘./content/csvdata2/cruise_ship_info.csv’

cruise_ship_info.cs 100%[===================>]   8.53K  --.-KB/s    in 0s      

2024-11-12 08:19:53 (51.5 MB/s) - ‘./content/csvdata2/cruise_ship_info.csv’ saved [8734/8734]



In [27]:
cruise_ship_data = sparkSession.read.csv(FILE_PATH2, sep=",", header=True, inferSchema=True)

In [28]:
cruise_ship_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [29]:
cruise_ship_data.groupBy("Cruise_line").count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [30]:
from pyspark.ml.feature import StringIndexer

In [31]:
indexer = StringIndexer(inputCol="Cruise_line",
                        outputCol="Cruise_category")
indexed = indexer.fit(cruise_ship_data).transform(cruise_ship_data)
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_category=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_category=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_category=1.0)]

In [32]:
cruise_assembler = VectorAssembler(inputCols=[ "Age",
                                               "Tonnage",
                                               "passengers",
                                               "length",
                                               "cabins",
                                               "passenger_density",
                                               "Cruise_category"],
                                   outputCol="features")

In [33]:
cruise_output = cruise_assembler.transform(indexed)

In [34]:
cruise_output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [35]:
cruise_final_data = cruise_output.select(["features", "crew"])
cruise_train_data, cruise_test_data = cruise_final_data.randomSplit([0.7, 0.3])

In [36]:
cruise_lr = LinearRegression(labelCol="crew")
cruise_lr_model = cruise_lr.fit(cruise_train_data)

24/11/12 08:20:02 WARN Instrumentation: [9fbbc84f] regParam is zero, which might cause numerical instability and overfitting.


In [37]:
cruise_lr_results = cruise_lr_model.evaluate(cruise_test_data)

In [38]:
cruise_lr_results.rootMeanSquaredError

0.7629264094842203

In [39]:
cruise_lr_results.r2

0.9412029924503627

In [40]:
cruise_lr_results.meanSquaredError

0.5820567062884843

In [41]:
cruise_lr_results.meanAbsoluteError

0.5660832088527805

In [42]:
from pyspark.sql.functions import corr

In [43]:
cruise_ship_data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [44]:
cruise_ship_data.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [45]:
cruise_ship_data.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)]